# torch.optim.Optimizer.state_dict()

- **state_dict()** нь оптимизаторын (Optimizer) төлөвийг төлөөлсөн dictionary буцаадаг. Энэ нь оптимизаторыг хадгалах, дуурайх, шилжүүлэх боломжийг олгодог.

## Үндсэн хэрэглээ

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Энгийн загвар ба оптимизатор үүсгэх
model = nn.Linear(10, 2)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Оптимизаторын төлөвийг авах
state = optimizer.state_dict()
print(f"State dict түлхүүрүүд: {list(state.keys())}")

State dict түлхүүрүүд: ['state', 'param_groups']


## State dict бүтэц
### Стандарт бүтэц

In [2]:
# Жишээ state dict бүтэц
{
    'state': {},          # Параметр бүрийн төлөв
    'param_groups': []    # Параметр бүлгүүдийн тохиргоо
}

{'state': {}, 'param_groups': []}

### Бодит жишээ

In [3]:
# Загвар үүсгэх
model = nn.Sequential(
    nn.Linear(5, 3),
    nn.ReLU(),
    nn.Linear(3, 1)
)

# SGD оптимизатор
optimizer_sgd = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# Adam оптимизатор
optimizer_adam = optim.Adam(model.parameters(), lr=0.001)

# State dict харьцуулах
sgd_state = optimizer_sgd.state_dict()
adam_state = optimizer_adam.state_dict()

print("SGD state dict түлхүүрүүд:")
print(list(sgd_state.keys()))

print("\nAdam state dict түлхүүрүүд:")
print(list(adam_state.keys()))

print("\nSGD param_groups тоо:", len(sgd_state['param_groups']))
print("Adam param_groups тоо:", len(adam_state['param_groups']))

SGD state dict түлхүүрүүд:
['state', 'param_groups']

Adam state dict түлхүүрүүд:
['state', 'param_groups']

SGD param_groups тоо: 1
Adam param_groups тоо: 1


## Нарийвчласан тайлбар
### 1. param_groups хэсэг

In [4]:
# Параметр бүлгүүдийг шалгах
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

state = optimizer.state_dict()
param_groups = state['param_groups']

print(f"Параметр бүлгийн тоо: {len(param_groups)}")

# Эхний параметр бүлгийн мэдээлэл
first_group = param_groups[0]
print("\nЭхний параметр бүлгийн мэдээлэл:")
for key, value in first_group.items():
    if key == 'params':
        print(f"  {key}: Параметр ID-уудын жагсаалт ({len(value)} ширхэг)")
    else:
        print(f"  {key}: {value}")

Параметр бүлгийн тоо: 1

Эхний параметр бүлгийн мэдээлэл:
  lr: 0.001
  betas: (0.9, 0.999)
  eps: 1e-08
  weight_decay: 0.01
  amsgrad: False
  maximize: False
  foreach: None
  capturable: False
  differentiable: False
  fused: None
  decoupled_weight_decay: False
  params: Параметр ID-уудын жагсаалт (4 ширхэг)


### 2. state хэсэг

In [5]:
# Загвар ба оптимизатор үүсгэх
model = nn.Linear(3, 2)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# Хэд хэдэн алхам хийх
for i in range(3):
    optimizer.zero_grad()
    loss = model(torch.randn(1, 3)).sum()
    loss.backward()
    optimizer.step()

# State dict авах
state_dict = optimizer.state_dict()

print("State хэсэгт байгаа параметр ID-ууд:")
for param_id in state_dict['state'].keys():
    print(f"  Параметр ID: {param_id}")
    
    # Параметр тус бүрийн төлөв
    param_state = state_dict['state'][param_id]
    print(f"    Төлөвийн түлхүүрүүд: {list(param_state.keys())}")
    
    if 'momentum_buffer' in param_state:
        print(f"    Momentum buffer хэлбэр: {param_state['momentum_buffer'].shape}")

State хэсэгт байгаа параметр ID-ууд:
  Параметр ID: 0
    Төлөвийн түлхүүрүүд: ['momentum_buffer']
    Momentum buffer хэлбэр: torch.Size([2, 3])
  Параметр ID: 1
    Төлөвийн түлхүүрүүд: ['momentum_buffer']
    Momentum buffer хэлбэр: torch.Size([2])


## Практик хэрэглээ
### 1. Оптимизаторыг хадгалах, дуурайх

In [6]:
def save_and_load_optimizer():
    """Оптимизаторыг хадгалах, дуурайх жишээ"""
    # 1. Анхны загвар, оптимизатор
    model1 = nn.Linear(5, 2)
    optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
    
    # Зарим сургалтын алхам хийх
    for _ in range(5):
        optimizer1.zero_grad()
        loss = model1(torch.randn(2, 5)).sum()
        loss.backward()
        optimizer1.step()
    
    # 2. State dict хадгалах
    saved_state = optimizer1.state_dict()
    
    # 3. Шинэ оптимизатор үүсгэж, state dict дуурайх
    model2 = nn.Linear(5, 2)
    optimizer2 = optim.Adam(model2.parameters(), lr=0.01)  # Өөр lr
    
    print("Дуурайхаас өмнөх lr:", optimizer2.param_groups[0]['lr'])
    
    # State dict дуурайх
    optimizer2.load_state_dict(saved_state)
    
    print("Дуурайсны дараах lr:", optimizer2.param_groups[0]['lr'])
    
    # Параметр ID-уудыг шалгах
    print(f"\nАнхны оптимизаторын параметр ID-ууд: {list(optimizer1.state_dict()['state'].keys())}")
    print(f"Шинэ оптимизаторын параметр ID-ууд: {list(optimizer2.state_dict()['state'].keys())}")
    
    return optimizer1, optimizer2

opt1, opt2 = save_and_load_optimizer()

Дуурайхаас өмнөх lr: 0.01
Дуурайсны дараах lr: 0.001

Анхны оптимизаторын параметр ID-ууд: [0, 1]
Шинэ оптимизаторын параметр ID-ууд: [0, 1]


### 2. Checkpoint хадгалах

In [7]:
def save_checkpoint(model, optimizer, epoch, path='checkpoint.pth'):
    """Checkpoint хадгалах"""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': 0.1  # Жишээ утга
    }
    
    torch.save(checkpoint, path)
    print(f"Checkpoint хадгаллаа: {path}")
    
    return checkpoint

def load_checkpoint(model, optimizer, path='checkpoint.pth'):
    """Checkpoint дуурайх"""
    checkpoint = torch.load(path)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    print(f"Checkpoint дуурайлаа: epoch {checkpoint['epoch']}")
    
    return checkpoint['epoch']

# Туршилт
model = nn.Sequential(nn.Linear(10, 5), nn.ReLU(), nn.Linear(5, 1))
optimizer = optim.RMSprop(model.parameters(), lr=0.01)

# Checkpoint хадгалах
save_checkpoint(model, optimizer, epoch=10)

# Шинэ загварт дуурайх
new_model = nn.Sequential(nn.Linear(10, 5), nn.ReLU(), nn.Linear(5, 1))
new_optimizer = optim.RMSprop(new_model.parameters(), lr=0.1)

epoch = load_checkpoint(new_model, new_optimizer)
print(f"Дуурайсан epoch: {epoch}")
print(f"Шинэ оптимизаторын lr: {new_optimizer.param_groups[0]['lr']}")

Checkpoint хадгаллаа: checkpoint.pth
Checkpoint дуурайлаа: epoch 10
Дуурайсан epoch: 10
Шинэ оптимизаторын lr: 0.01
